#### importing libraries

In [3]:
import pandas as pd
import xlsxwriter
import datetime
import yfinance as yf

<h4>Reading data from Master file and classify</h4>

In [5]:
excel_file = r"Master Stock Data.xlsx"
df = pd.read_excel(excel_file,engine='openpyxl')
buy_df = df[df['Buy/Sell'] == 'B'].reset_index(drop=True)
sell_df = df[df['Buy/Sell'] == 'S'].reset_index(drop=True)
#buy_df

#### initialising sample resultant_df

In [7]:
resultant_df =  pd.DataFrame(columns = [
    "S.No",
    "Entry Date",       #? Trade Date aka Buy date in Equity Tradelisting sheet
    "Stocks",           #? Scrip Symbol - E.g : AJANTPHARM
    "Transaction",      #? Buy or Sell - here in this column, it's BUY only
    "Buy Quantity",     #? Buying Quantity  - column F  in BUY Category 
    "Entry Price",      #? Rate - Column G in BUY Category
    "Buy Cost",         #? Brokerage + (IGST + SGST + CGST + UTGST + Cess + Stamp + TO Charges + Sebi Fees + STT Amount)
    "Buy Value",        #? Buy Quantity * Entry Price + Buy Cost
    
    "Exit Date",        #? Sell Date in respective Sell Category --> look at immediate sold stock for respective stock
    "Sell Quantity",    #? Sell Quantity (need not be same as entry)
    "Exit Price",       #? Rate - Column G in SELL Category
    "Sell Cost",        #? Brokerage + (IGST + SGST + CGST + UTGST + Cess + Stamp + TO Charges + Sebi Fees + STT Amount)
    "Sell value",       #? Sell Quantity * Exit Price + Sell Cost
    
    "PNL",              #? Profit or Loss : Exit Price - Entry Price
    "Net PNL",          #? Sell Value - Buy Value
    "Percentage",       #? Net PNL / Buy Value
    "Cumulative Profit",#? We have to sum the Net PNL for every past row and current row and make that as CP
])

#### adding initial buy data to resultant_df

In [8]:
resultant_df["Entry Date"] = buy_df["Trade Date"]
resultant_df["Stocks"] = buy_df["Scrip Symbol"]
resultant_df["Transaction"] = "Buy"
resultant_df["Buy Quantity"] = buy_df["Quantity"]
resultant_df["Entry Price"] = buy_df["Rate"]
resultant_df["Buy Cost"] = buy_df["Brokerage"] + buy_df["IGST"] + buy_df["SGST"] + buy_df["CGST"]\
                            + buy_df["UTGST"] + buy_df["Cess"] + buy_df["Stamp"] + buy_df["TO Charges"] + buy_df["Sebi Fees"]\
                            + buy_df["STT Amount"]
resultant_df["Buy Value"] = (resultant_df["Buy Quantity"] * resultant_df["Entry Price"]) + resultant_df["Buy Cost"]

#### function for one buy and one sell

In [9]:
def fill_data_for_single_buy_single_sell(stock, individual_stock_sell_df):
    temp_sell_df    = individual_stock_sell_df.iloc[0]
    exit_date       = temp_sell_df["Trade Date"]
    sell_quantity   = temp_sell_df["Quantity"]
    exit_price      = temp_sell_df["Rate"]
    sell_cost       = temp_sell_df["Brokerage"] + temp_sell_df["IGST"] + temp_sell_df["SGST"] + temp_sell_df["CGST"]\
                        + temp_sell_df["UTGST"] + temp_sell_df["Cess"] + temp_sell_df["Stamp"] + temp_sell_df["TO Charges"]\
                        + temp_sell_df["Sebi Fees"] + temp_sell_df["STT Amount"]
                    
    resultant_df.loc[resultant_df['Stocks'] == stock,
                    [
                    "Exit Date",
                    "Sell Quantity",
                    "Exit Price",
                    "Sell Cost"
                    ]
                ] = [
                        exit_date,
                        sell_quantity,
                        exit_price,
                        sell_cost
                    ]

#### code till single buy and single sell

In [10]:
total_stock_types = list(set(resultant_df["Stocks"].to_list()))

for stock in total_stock_types:
    individual_stock_buy_df = resultant_df[resultant_df["Stocks"] == stock]
    if len(individual_stock_buy_df) == 1:
        individual_stock_sell_df = sell_df[sell_df["Scrip Symbol"] == stock]
        if len(individual_stock_sell_df) == 1:
            if individual_stock_buy_df.iloc[0]["Buy Quantity"] == individual_stock_sell_df.iloc[0]["Quantity"]:
                fill_data_for_single_buy_single_sell(stock,individual_stock_sell_df)

In [12]:
resultant_df

,S.No,Entry Date,Stocks,Transaction,Buy Quantity,Entry Price,Buy Cost,Buy Value,Exit Date,Sell Quantity,Exit Price,Sell Cost,Sell value,PNL,Net PNL,Percentage,Cumulative Profit
0,NaN,2020-02-10,AJANTPHARM,Buy,36,1370.00,199.76,49519.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2020-02-25,AJANTPHARM,Buy,40,1370.00,199.76,54999.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2020-02-04,AMBER,Buy,31,1595.00,200.26,49645.26,2020-02-17 00:00:00,31,1516.0,190.33,NaN,NaN,NaN,NaN,NaN
3,NaN,2020-02-07,APLLTD,Buy,77,649.64,202.60,50224.88,2020-02-24 00:00:00,77,643.5,200.68,NaN,NaN,NaN,NaN,NaN
4,NaN,2019-10-18,BAJFINANCE,Buy,77,4132.00,200.79,318364.79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,2020-03-03,BANKBEES,Buy,77,297.45,154.31,23057.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,2020-01-21,CANFINHOME,Buy,77,414.80,201.58,32141.18,2020-01-22 00:00:00,77,477.2,231.9,NaN,NaN,NaN,NaN,NaN
7,NaN,2020-01-20,CUPID,Buy,77,257.19,202.05,20005.68,2020-01-22 00:00:00,77,228.25,179.32,NaN,NaN,NaN,NaN,NaN
8,NaN,2019-10-09,DABUR,Buy,77,443.60,203.13,34360.33,2020-02-17 00:00:00,77,501.68,229.58,NaN,NaN,NaN,NaN,NaN
9,NaN,2020-02-26,DEEPAKNTR,Buy,77,509.97,202.37,39470.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
